In [1]:
! pip install qutip qutip_qip matplotlib qiskit qutip-qtrl tqdm

In [2]:
from qiskit import QuantumCircuit
from qiskit.visualization import plot_histogram
from qutip_qip.qiskit import QiskitCircuitSimulator,QiskitPulseSimulator

import numpy as np
import matplotlib.pyplot as plt
import qutip
from qutip import basis, fidelity,sigmax, sigmay, sigmaz,identity,rand_unitary,tensor,mesolve,Qobj, Bloch
from qutip.qip.operations import  rx,ry,rz,cz_gate
from typing import List
from tqdm import tqdm
from qutip_qip.device import Processor, Model
from qiskit import QuantumCircuit
from qiskit.compiler import transpile
import qiskit


## Exercise 1

In [3]:
GROUND = basis(4, 0)
EXCITED = basis(4, 1)
LEAKAGE = basis(4, 2)
RYDBERG = basis(4, 3)

class minimal_processor(Processor):

    basis_gates=['rx','rz','cz']
    qbt_dim = 4
    
    def __init__(self,num_qubits=2,gamma_r = 1 / 540):
        
        self.model=Model(num_qubits=num_qubits,
                         dims=[self.qbt_dim for _ in range(num_qubits)])
        self.num_qubits=num_qubits
        self.get_c_ops(gamma_r)

    def evolve(self,
               state:qutip.Qobj,
                ins: qiskit._accelerate.circuit.CircuitInstruction):
        name = ins.operation.name
        param = next(iter(ins.operation.params), None)
        qbts = [qubit._index for qubit in ins.qubits]

        if name == 'rx':
            Omega_01 = 1
            return mesolve(
                    H=self.id_wrap(Omega_01/2 * (GROUND * EXCITED.dag() + EXCITED * GROUND.dag()),
                                   qbts[0]), 
                    rho0=state, 
                    tlist=np.array([0, param[0]/Omega_01]), 
                    c_ops=self.collapse_ops, 
                    options={'store_final_state':True}
                )
        elif name == 'rz':
            delta_1 = 1
            return mesolve(
                    H=self.id_wrap(delta_1 * EXCITED * EXCITED.dag(),
                                   qbts[0]), 
                    rho0=state, 
                    tlist=np.array([0,param[0]/delta_1]), 
                    c_ops=self.collapse_ops, 
                    options={'store_final_state':True}
                )
        elif name == 'cz':
            t_tot = 0.540 #microsecond, which is the total duration of the gate protocol
            B=200*2*np.pi #interaction strength
            omegaMax = 17*2*np.pi #Mhz
            deltaMax = 23*2*np.pi #Mhz
            tau = 0.175*t_tot 
            a = np.exp(-(t_tot/4)**4/tau**4)
            def Rabi_frequency(t,):
                return (t<t_tot/2)*omegaMax*(np.exp(-(t-t_tot/4)**4/tau**4)-a)/(1-a)    \
                +    (t>=t_tot/2)*omegaMax*(np.exp(-(t-3*t_tot/4)**4/tau**4)-a)/(1-a)
            def Detunning(t):
                return (t<t_tot/2)*(-1)*deltaMax*np.cos(2*np.pi*t/t_tot) \
                        +    (t>=t_tot/2)*deltaMax*np.cos(2*np.pi*t/t_tot)
            H = []
            for idx in qbts:
                H.append([self.id_wrap(0.5*RYDBERG*EXCITED.dag()+0.5*EXCITED*RYDBERG.dag(), 
                                       idx),
                          Rabi_frequency])
                H.append([self.id_wrap(RYDBERG*RYDBERG.dag(), 
                                       idx),
                          Detunning])
                
            H.append(
                [B*tensor(RYDBERG,RYDBERG)*tensor(RYDBERG,RYDBERG).dag()]
            ) # This assumes the system involves two qubit only.

            return mesolve(
                    H=H, 
                    rho0=state, 
                    tlist=np.array([0, t_tot]), 
                    c_ops=self.collapse_ops, 
                    options={'store_final_state':True}
                )
            
    
    def run_state(self,
                  init_state:qutip.Qobj) -> qutip.solver.Result:
        if init_state.isket:
            assert init_state.dims == [[self.qbt_dim for _ in range(self.num_qubits)],[1 for _ in range(self.num_qubits)]]
        else:
            assert init_state.dims == [[self.qbt_dim for _ in range(self.num_qubits)],[self.qbt_dim for _ in range(self.num_qubits)]]
        state = init_state
        for ins in tqdm(self.qiskit_circ_transpiled.data,'Simulating gate'):
            result = self.evolve(state,ins)
            state = result.final_state
        return result
    
    def generate_init_processor_state(self)->qutip.Qobj:
        # Always initialize in zero
        return tensor([basis(self.qbt_dim, 0) for _ in range(self.num_qubits)])
    
    def id_wrap(self,op,idx):
        return tensor([qutip.identity(self.qbt_dim) for _ in range(0,idx)]+[op]+[qutip.identity(self.qbt_dim) for _ in range(idx+1,self.num_qubits)])
    
    def get_c_ops(self,gamma_r):
        L0 = np.sqrt(1/16 * gamma_r) * (GROUND * RYDBERG.dag())
        L1 = np.sqrt(1/16 * gamma_r) * (EXCITED * RYDBERG.dag())
        Ld = np.sqrt(7/8 * gamma_r) * (LEAKAGE * RYDBERG.dag())
        single_q_c_ops = [L0, L1, Ld]

        self.collapse_ops = []
        for q in range(self.num_qubits):
            for c_op in single_q_c_ops:
                self.collapse_ops.append(self.id_wrap(c_op,q))
    
    def load_circuit(self,
                     qutip_circuit:qiskit.QuantumCircuit)->None:
        self.qiskit_circ_transpiled = transpile(qutip_circuit,basis_gates=self.basis_gates)
    
    def get_final_circuit_state(self,result:qutip.solver.Result)->qutip.Qobj:
        rho = result.final_state
        if rho.isket:
            rho = qutip.ket2dm(rho)
        rho_arr = rho.full()
        rho_2lvl=rho_arr.reshape(4,4,4,4)[:2,:2,:2,:2]
        return Qobj(rho_2lvl.reshape(4,4),dims=[[2,2],[2,2]])

In [4]:
processor = minimal_processor(2)
pulse_backend = QiskitPulseSimulator(processor)

In [5]:
circ1 = QuantumCircuit(2)
circ1.h(0)
circ1.h(1)
job = pulse_backend.run(circ1)
result = job.result()
result.data()['statevector'].data 
result.data()['counts']

AttributeError: 'QubitCircuit' object has no attribute 'qubits'

In [6]:
from qutip_qip.qiskit.converter import convert_qiskit_circuit

qutip_circ = convert_qiskit_circuit(
            circ1,
            allow_custom_gate=pulse_backend.options.allow_custom_gate,
        )


In [11]:
qutip_circ.__repr__()

'<qutip_qip.circuit.circuit.QubitCircuit object at 0x12b6ff160>'

In [ ]:
dir(qutip_circ)

['N',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_raw_img',
 '_to_qasm',
 'add_1q_gate',
 'add_circuit',
 'add_gate',
 'add_gates',
 'add_measurement',
 'add_state',
 'adjacent_gates',
 'compute_unitary',
 'dims',
 'draw',
 'gates',
 'input_states',
 'latex_code',
 'name',
 'num_cbits',
 'output_states',
 'png',
 'propagators',
 'remove_gate_or_measurement',
 'resolve_gates',
 'reverse_circuit',
 'reverse_states',
 'run',
 'run_statistics',
 'svg',
 'user_gates']

In [ ]:
backend = QiskitCircuitSimulator()


In [ ]:
circ2 = QuantumCircuit(2)
circ2.h(0)
circ2.cx(0,1)

In [ ]:
circ3 = QuantumCircuit(3)
circ3.h(0)
circ3.cx(0,1)
circ3.cx(1,2)